<i>Copyright (c) Microsoft Corporation. All rights reserved.</i>

<i>Licensed under the MIT License.</i>

# NPA: Neural News Recommendation with Personalized Attention
NPA \[1\] is a news recommendation model with personalized attention. The core of NPA is a news representation model and a user representation model. In the news representation model we use a CNN network to learn hidden representations of news articles based on their titles. In the user representation model we learn the representations of users based on the representations of their clicked news articles. In addition, a word-level and a news-level personalized attention are used to capture different informativeness for different users.

## Properties of NPA:
- NPA is a content-based news recommendation method.
- It uses a CNN network to learn news representation. And it learns user representations from their clicked news articles.
- A word-level personalized attention is used to help NPA attend to important words for different users.
- A news-level personalized attention is used to help NPA attend to important historical clicked news for different users.

## Data format:

### train data
One simple example: <br>

`1 0 0 0 0 Impression:0 User:2903 CandidateNews0:27006,11901,21668,9856,16156,21390,1741,2003,16983,8164 CandidateNews1:8377,10423,9960,5485,20494,7553,1251,17232,4745,9178 CandidateNews2:1607,26414,25830,16156,15337,16461,4004,6230,17841,10704 CandidateNews3:17323,20324,27855,16156,2934,14673,551,0,0,0 CandidateNews4:7172,3596,25442,21596,26195,4745,17988,16461,1741,76 ClickedNews0:11362,8205,22501,9349,12911,20324,1238,11362,26422,19185 ...`
<br>

In general, each line in data file represents one positive instance and n negative instances in a same impression. The format is like: <br>

`[label0] ... [labeln] [Impression:i] [User:u] [CandidateNews0:w1,w2,w3,...] ... [CandidateNewsn:w1,w2,w3,...] [ClickedNews0:w1,w2,w3,...] ...`

<br>

It contains several parts seperated by space, i.e. label part, Impression part `<impresison id>`, User part `<user id>`, CandidateNews part, ClickedHistory part. CandidateNews part describes the target news article we are going to score in this instance, it is represented by (aligned) title words. To take a quick example, a news title may be : `Trump to deliver State of the Union address next week` , then the title words value may be `CandidateNewsi:34,45,334,23,12,987,3456,111,456,432`. <br>
ClickedNewsk describe the k-th news article the user ever clicked and the format is the same as candidate news. Words are aligned in news title. We use a fixed length to describe an article, if the title is less than the fixed length, just pad it with zeros.

### test data
One simple example: <br>
`1 Impression:0 User:6446 CandidateNews0:18707,23848,13490,10948,21385,11606,1251,16591,827,28081 ClickedNews0:27838,7376,16567,28518,119,21248,7598,9349,20324,9349 ClickedNews1:7969,9783,1741,2549,27104,14669,14777,21343,7667,20324 ...`
<br>

In general, each line in data file represents one instance. The format is like: <br>

`[label] [Impression:i] [User:u] [CandidateNews0:w1,w2,w3,...] [ClickedNews0:w1,w2,w3,...] ...`
<br>

## Global settings and imports

In [1]:
import sys
sys.path.append("../../")
from reco_utils.recommender.deeprec.deeprec_utils import download_deeprec_resources 
from reco_utils.recommender.newsrec.newsrec_utils import prepare_hparams
from reco_utils.recommender.newsrec.models.npa import NPAModel
from reco_utils.recommender.newsrec.io.news_iterator import NewsIterator
import papermill as pm
from tempfile import TemporaryDirectory
import tensorflow as tf
import os

print("System version: {}".format(sys.version))
print("Tensorflow version: {}".format(tf.__version__))

tmpdir = TemporaryDirectory()

System version: 3.6.10 |Anaconda, Inc.| (default, May  8 2020, 02:54:21) 
[GCC 7.3.0]
Tensorflow version: 1.15.2


## Download and load data

In [2]:
data_path = tmpdir.name
yaml_file = os.path.join(data_path, r'npa.yaml')
train_file = os.path.join(data_path, r'train.txt')
valid_file = os.path.join(data_path, r'test.txt')
wordEmb_file = os.path.join(data_path, r'embedding.npy')
if not os.path.exists(yaml_file):
    download_deeprec_resources(r'https://recodatasets.blob.core.windows.net/newsrec/', data_path, 'npa.zip')

100%|██████████| 21.2k/21.2k [00:01<00:00, 14.0kKB/s]


## Create hyper-parameters

In [3]:
epochs=5
seed=42

In [4]:
hparams = prepare_hparams(yaml_file, wordEmb_file=wordEmb_file, epochs=epochs)
print(hparams)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

data_format=news,iterator_type=None,wordEmb_file=/tmp/tmph3zpuc6n/embedding.npy,doc_size=10,title_size=None,body_size=None,word_emb_dim=100,word_size=28929,user_num=10338,vert_num=None,subvert_num=None,his_size=50,npratio=4,dropout=0.2,attention_hidden_dim=200,head_num=4,head_dim=100,cnn_activation=relu,dense_activation=None,filter_num=400,window_size=3,vert_emb_dim=100,subvert_emb_dim=100,gru_unit=400,type=ini,user_emb_dim=50,learning_rate=0.0001,loss=cross_entropy_loss,optimizer=adam,epochs=5,batch_size=64,show_step=100000,metrics=['group_auc', 'mean_mrr', 'ndcg@5;10']


In [5]:
iterator = NewsIterator

## Train the NPA model

In [6]:
model = NPAModel(hparams, iterator, seed=seed)


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.



In [7]:
print(model.run_eval(valid_file))


{'group_auc': 0.51, 'mean_mrr': 0.163, 'ndcg@5': 0.1522, 'ndcg@10': 0.2123}


In [8]:
model.fit(train_file, valid_file)

at epoch 1
train info: logloss loss:1.6397553108176406
eval info: group_auc:0.538, mean_mrr:0.1686, ndcg@10:0.2298, ndcg@5:0.1625
at epoch 1 , train time: 13.9 eval time: 8.9
at epoch 2
train info: logloss loss:1.590539211156417
eval info: group_auc:0.5392, mean_mrr:0.1737, ndcg@10:0.2335, ndcg@5:0.1673
at epoch 2 , train time: 12.5 eval time: 8.7
at epoch 3
train info: logloss loss:1.5472524715929614
eval info: group_auc:0.5517, mean_mrr:0.1739, ndcg@10:0.2385, ndcg@5:0.1659
at epoch 3 , train time: 12.6 eval time: 8.8
at epoch 4
train info: logloss loss:1.4370532391022663
eval info: group_auc:0.5575, mean_mrr:0.1741, ndcg@10:0.2411, ndcg@5:0.1676
at epoch 4 , train time: 12.4 eval time: 8.9
at epoch 5
train info: logloss loss:1.3469764787323621
eval info: group_auc:0.5609, mean_mrr:0.1783, ndcg@10:0.2486, ndcg@5:0.1697
at epoch 5 , train time: 12.4 eval time: 8.7


In [9]:
res_syn = model.run_eval(valid_file)
print(res_syn)
pm.record("res_syn", res_syn)

{'group_auc': 0.5609, 'mean_mrr': 0.1783, 'ndcg@5': 0.1697, 'ndcg@10': 0.2486}


/data/anaconda/envs/reco_gpu/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Function record is deprecated and will be removed in verison 1.0.0 (current version 0.19.1). Please see `scrapbook.glue` (nteract-scrapbook) as a replacement for this functionality.
  This is separate from the ipykernel package so we can avoid doing imports until


## Reference
\[1\] Chuhan Wu, Fangzhao Wu, Mingxiao An, Jianqiang Huang, Yongfeng Huang and Xing Xie: NPA: Neural News Recommendation with Personalized Attention, KDD 2019, ADS track.<br>